                                                                                                                    SBC - GIA
___

<div style="text-align: center;">
  <span style="font-family: 'Playfair Display', serif; font-size: 24px; font-weight: bold;">
Implementation Decision Tree Python
  </span>
</div>

___


This notebook documents the process of creating a decision tree from scratch. The focus lies in how to implement each stage of the process. 

**Index**
- Dataset Review.
    - Preprocessing.
- Code.
    - Classes and Functions.
    - Main.

##### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter
from collections import defaultdict
from graphviz import Digraph

import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

from cas import Cas

<div style="background-color:#F2F2F2; padding: 10px;">
    <div style="text-align: center;">
        <span style="font-family: 'Playfair Display', serif; font-size: 20px; font-weight: bold;">
            1. Dataset Review
        </span>
    </div>
</div>



In this section, we focus on loading the dataset and conducting a preliminary examination. 

In [2]:
data = pd.read_csv('../ontologia/Cases.csv') # ../ontologia/

data.head()

,id_usuari,id_llibre,score,genere_persona,any_naixement,pref_adaptacio_peli,pref_best_seller,pref_tipus_lectura,pref_sagues,comprat,...,Romance,Ciencia_Ficcio,Comedia,Historica,Fantasia,Ciencia,Creixement_personal,Policiaca,Juvenil,pagines_max
0,0,43,4.45,Dona,2003,Si,No,Indiferent,No,Si,...,1,1,1,1,0,0,0,0,0,350
1,0,97,4.50,Dona,2003,Si,No,Indiferent,No,Si,...,1,1,1,1,0,0,0,0,0,350
2,0,16,4.85,Dona,2003,Si,No,Indiferent,No,Si,...,1,1,1,1,0,0,0,0,0,350
3,0,84,4.70,Dona,2003,Si,No,Indiferent,No,Si,...,1,1,1,1,0,0,0,0,0,350
4,0,4,4.90,Dona,2003,Si,No,Indiferent,No,Si,...,1,1,1,1,0,0,0,0,0,350


Upon reviewing the dataset, we observe that it comprises the following columns, each representing distinct data attributes:

|   COLUMS             |          DATASET   |
|----------------------|--------------------|
| genere_persona       | Fantasia           |
| any_naixement        | Ciencia            |
| pref_adaptacio_peli  | Creixement_personal|
| pref_best_seller     | Autoajuda          |
| pref_sagues          | Policiaca          |
| Comedia              | Juvenil            |
| Historica            | Densa              |
| Romance              | Fluida             |
| Ciencia_Ficcio       | pagines_max        |
| Ficcio               |                    |



A decision tree is a machine learning algorithm that works by dividing data at each node based on a variable corresponding to each column in the dataset. The tree branches out into as many paths as there are unique values in that variable, allowing it to make decisions and predictions based on these splits.

To do so, it was necessary to make a preprocessing of the de data. The preprocessing was made by knowing the importance of each feature, which was analise at the `Feature Decision.ipynb`. 

<div style="background-color:#F2F2F2; padding: 10px;">
    <div style="text-align: center;">
        <span style="font-family: 'Playfair Display', serif; font-size: 20px; font-weight: bold;">
            2. Code
        </span>
    </div>
</div>

In this section we present the code that has implemented, we can see the following classes and function: 
- Class `CaseTree`.
- Class `DecisionNode`.
- Function `build_tree`.
- Function `print_tree`.
- Function `plot_node`.
- Function `plot_tree`.

#### Class

In [3]:
class DecisionNode:
    def __init__(self, value=None, subset=None, path=None):
        """
        Initializes a DecisionNode.
            :param value: The The feature/question/cases subset associated with this node
            :param subset: The subset of Cas instances at this node. Only different from {} at leaf nodes.
            :param parent: DecisionNode instance being the parent of the actual DecisionNode instance. None if DecisionNode instance is root of the entire tree.
            :param path: The path taken to reach this node.
        """

        self.value = value
        self.children = {}
        self.subset = subset
        self.path = path or {}

    def add_child(self, answer, child):
        """
        Adds a child node associated with a specific answer.
            :param answer: The answer or value leading to the child node.
            :param child: The child node to add.
        """
        self.children[answer] = child

    def is_leaf(self):
        """
        Determines if this node is a leaf node (i.e., has no children).
            :return: True if the node is a leaf node, False otherwise.
        """
        return len(self.children) == 0
    
    def get_child(self, answer):
        """
        Retrieves a child node corresponding to a given answer.
            :param answer: The answer or value for which the child node is needed.
            :return: The child node associated with the given answer.
        """
        return self.children.get(answer)
    
    def get_leaves(self):
        
        """
        Retrieves the whole set of leaf nodes that hung from the actual DecisionNode instance
        as if it was the root of a subtree.
        """
        set_leaves = []
        def aux(node):
            if node.is_leaf():
                set_leaves.append(node)
            else:
                for child in node.children.values():
                    aux(child)
        aux(self)
        return set(set_leaves)

    def save(self, filename):
        """
        Saves the current state of the decision tree to a file.
            :param filename: The name of the file to save the tree.
        """
        with open(filename + '.pkl', 'wb') as file:
            pickle.dump(self, file)
    
    def load(self, filename):
        """
        Loads a decision tree from a file.
            :param filename: The name of the file to load the tree from.
            :return: The root node of the loaded decision tree.
        """
        with open(filename + '.pkl', 'rb') as file:
            return pickle.load(file)

In [8]:
class DecisionTree:
    def __init__(self, dataset, max_depth=6):
        self.max_depth = max_depth
        self.root = None
        self.dataset = dataset
        self.list_parents = []

    def build_tree(self, features, data, path={}, depth=0):
        """
        Builds a decision tree recursively.
            :param features: A list of features to use for splitting the data.
            :param data: The dataset to build the tree from.
            :param path: A dictionary representing the path taken to reach the current node.
            :param depth: The current depth in the tree.
            :param max_depth: The maximum depth of the tree.
            :return: The root node of the decision tree.
        """
        # Base case: Return a leaf node if maximum depth is reached, no features left, or no data
        #            The leaf have a set of Case instances that meet the attributs
        if depth == self.max_depth or len(features) == 0 or len(data) == 0:
            subset_cases, list_cases = set(), []
            
            for index, row in data.iterrows():
                problem_description = (row["genere_persona"], row["any_naixement"], row["pref_adaptacio_peli"], row["pref_best_seller"],
                                       row["pref_tipus_lectura"],row["pref_sagues"], row["Comedia"], row["Ficcio"], row["Ciencia_Ficcio"],
                                       row["Historica"], row["Romance"], row["Fantasia"], row["Ciencia"], row["Creixement_personal"],
                                       row["Policiaca"], row["Juvenil"], row["pagines_max"], row["comprat"])
                
                subset_cases.add(Cas(index, row["id_usuari"], problem_description, row["id_llibre"], row["score"]))
                list_cases.append(index)

            value = f"Subset Cases:\n {list_cases}"
            return DecisionNode(value=value, subset=subset_cases, path=path)
        
        # Select the current feature to split on.
        #     `any_naixement` requires specific treatment because we are looking for an interval
        
        current_feature = features[0]
        node = DecisionNode(value=current_feature, path=path)

        if current_feature != 'any_naixement':
            possible_answers = self.dataset[current_feature].unique()
            
            for answer in possible_answers:
                # Create a subset of data corresponding to the current answer
                subset = data[data[current_feature] == answer]
                
                remaining_features = features[1:]
                new_path = path.copy()
                new_path[current_feature] = str(answer)
                
                # Recursively call `build_tree` to build the subtree
                child_node = self.build_tree(remaining_features, subset, new_path, depth + 1)
                node.add_child(answer, child_node)
        else:
            # Based on the stadistic study, we use 2003 to split `any_naixement`
            subset_1 = data[data[current_feature] < 2003]
            subset_2 = data[data[current_feature] >= 2003]
            remaining_features = features[1:]

            new_path_1 = path.copy(); new_path_2 = path.copy()
            new_path_1[current_feature] = "< 2003"
            new_path_2[current_feature] = ">= 2003"
                
            # Recursively call `build_tree` to build the subtree twice.
            child_node_1 = self.build_tree(remaining_features, subset_1, new_path_1, depth + 1)
            child_node_2 = self.build_tree(remaining_features, subset_2, new_path_2, depth + 1)
            node.add_child("< 2003", child_node_1)
            node.add_child(">= 2003", child_node_2)

        return node

    def print_tree(self, node=None, depth=0):
        """
        Prints a representation of the decision tree.
            :param node: The current node in the tree.
            :param depth: The depth of the current node, used for indentation.
        """
        if node.is_leaf():  # If it's a leaf node, print its value
            print("  " * depth + "Leaf:", node.value)
        else:  # If it's a decision node
            print("  " * depth + "Question:", node.value)
            for answer, child in node.children.items():  # Iterate through children
                print("  " * (depth + 1) + "Answer:", answer)
                self.print_tree(child, depth + 2)  # Recursively print each child node

    def plot_node(self, graph, node, node_id, parent_id=None, label=None):
        """
        Plots a single node and its connections in the decision tree graph.
            :param graph: The graph object to which nodes and edges are added.
            :param node: The current node to plot.
            :param node_id: The unique identifier for the current node.
            :param parent_id: The identifier of the parent node.
            :param label: The label for the edge connecting to the parent node.
        """
        if node.is_leaf():  # If it's a leaf node
            graph.node(node_id, label=node.value, shape="box")
            if parent_id is not None:
                graph.edge(parent_id, node_id, label=str(label))
        
        else:  # If it's a decision node
            graph.node(node_id, label=node.value)
            if parent_id is not None:
                graph.edge(parent_id, node_id, label=str(label))
            
            for answer, child in node.children.items():  # Iterate through children
                child_id = f"{node_id}_{str(answer)}"
                self.plot_node(graph, child, child_id, node_id, label=answer)

    def plot_tree(self):
        """
        Creates a graph representation of the decision tree.
            :param root: The root node of the decision tree.
            :return: The graph object representing the tree.
        """
        graph = Digraph(comment='Decision Tree')
        self.plot_node(graph, self.root, 'root')
        return graph

    def evaluate_case_through_tree(self, node, case):
        """
        Evaluates a case through the decision tree to find the corresponding leaf node.
            :param node: The current node in the tree.
            :param case: The case to be evaluated, represented as a dictionary of features.
            :returns: 1) The leaf node corresponding to the case, or None if no valid path is found.
                      2) The list of DecisionNode instances that are the ancestors of this leaf node.
        """
        if node.is_leaf():
            return node, self.list_parents

        feature_to_check = node.value # The feature that the current node is querying
        feature_value_in_case = case.get(feature_to_check)

        # If the feature is not in the case, evaluation cannot proceed
        if feature_value_in_case is None:
            return None

        # Find the child node corresponding to the answer in the case
        child_node = node.get_child(feature_value_in_case)

        # If there is no corresponding child node, the path is invalid
        if child_node is None:
            return None

        self.list_parents += [child_node]
        return self.evaluate_case_through_tree(child_node, case)

    def make_trace(self, node, new_case):
        """
        Traces and prints the path taken for a given case in the decision tree.
            :param decision_tree_root: The root node of the decision tree.
            :param new_case: The new case to be traced, represented as a dictionary of features.
        """
        leaf_node = self.evaluate_case_through_tree(node, new_case)
        print(leaf_node)
        # Trace and print the path:
        print("This case has been categorized in this leaf as they share the following attributes:")
        for attribute, answer in leaf_node.path.items():
            print(f"       - The attribute {attribute} has an answer of {str(answer)}.")


#### Main

Below is provided a basic example to understand how to use it:
```
    1. Data and Features:
        data = pd.read_csv('') 
        features_ordered_by_importance = ['feature1', 'feature2', 'feature3']

    2. Create the DT:
        dt = DecisionTree(max_depth=6)  # Adjust max_depth if needed
        decision_tree_root = dt.build_tree(features_ordered_by_importance, data)
        dt.root = decision_tree_root  # Set the root of the tree

    3. DT Graphic (Optional):
        graph = dt.plot_tree()
        graph.render('decision_tree', view=True)
    
    5. New case to be classified:
        case_example = {'feature1': 'answer1',
                        'feature2': 'answer2',
                        'feature3': 'answer3'}

    6. Make a Trace: 
        dt.make_trace(dt.root, case_example)
```

In [9]:
# 1. Data and Features
features_ordered_by_importance = ['genere_persona', 'pref_sagues', 'pref_tipus_lectura', 'Romance', 'Ciencia_Ficcio', 'any_naixement', 
                                  'Comedia', 'Historica', 'Ficcio', 'Fantasia', 'Ciencia', 'Creixement_personal', 'Policiaca', 'Juvenil', 
                                  'pagines_max', 'pref_adaptacio_peli', 'pref_best_seller']

# 2. Create the DT
dt = DecisionTree(data, max_depth=6)  # Adjust max_depth if needed
decision_tree_root = dt.build_tree(features_ordered_by_importance, data)
dt.root = decision_tree_root  # Set the root of the tree

# 3. DT Graphic (Optional)
graph = dt.plot_tree()
graph.render('decision_tree', view=True)

# 4. New case to be classified:
case1 = data.iloc[60]

# 5. Make Trace
dt.make_trace(dt.root, case1)

None
This case has been categorized in this leaf as they share the following attributes:


AttributeError: 'NoneType' object has no attribute 'path'

In [ ]:
# SANITY CHECK
d = data[data['Romance'] == 1]
d = d[d['pref_sagues'] == 'No']
d = d[d['pref_tipus_lectura'] == 'Fluida']
d = d[d['pref_best_seller'] == 'Si']
d = d[d['pref_adaptacio_peli'] == 'Si']
d = d[d['genere_persona'] == 'Dona']
d

#### Save Model

To save the model, you need to define the path where you want it to be saved, including the filename. 

It is worth saying that when saving the decision tree model, ensure that the model is not converted into a graphical format.

In [ ]:
PATH_FILE = '../Code/Decision_Tree.pkl'
decision_tree_root.save(PATH_FILE)

In case of wanting to load the model, it is necessary to do the following proces: 

```python
# Load the Model
dt = DecisionNode()

dt = dt.load(PATH_FILE)
```
---
---